# Small System

There is a single process to be optimized for cost [USD] over 4 quarters of a year. 
There is variability in terms of how much of the (known) process [Wind Farm] capacity can be accessed, and resource (Power) demand.


## Initialize

In [1]:
from energia import *

m = Model('example1')

## Time

We have 4 quarter which form a year

In [2]:
m.q = Periods()
m.y = 4 * m.q

The horizon ($\overset{\ast}{t} = argmin_{n \in N} |\mathcal{T}_{n}|$) is determined by Energia implicitly. Check it by printing Model.Horizon

In [3]:
m.horizon

y

## Space

If nothing is provided, a default location is created. The created single location will serve as the de facto network

In single location case studies, it may be easier to skip providing a location all together 

In [4]:
m.network

l

## Resources

In the Resource Task Network (RTN) methodology all commodities are resources. In this case, we have a few general resources (wind, power) and a monetary resource (USD).

In [5]:
m.usd = Currency()
m.power, m.wind = Resource(), Resource()

### Setting Bounds

The first bound is set for over the network and year, given that no spatiotemporal disposition is provided: 

$\mathbf{cons}_{wind, network, year_0} \leq 400$ 

For the second bound, given that a nominal is given, the list is treated as multiplicative factor. The length matches with the quarterly scale. Thus:

$\mathbf{rlse}_{power, network, quarter_0} \geq 60$

$\mathbf{rlse}_{power, network, quarter_1} \geq 70$

$\mathbf{rlse}_{power, network, quarter_2} \geq 100$

$\mathbf{rlse}_{power, network, quarter_3} \geq 30$


In [6]:
m.wind.consume <= 400
m.power.release.prep(100) >= [0.6, 0.7, 1, 0.3]

--- General Resource Balance for wind in (l, y): initializing constraint, adding consume(wind, l, y)
    Completed in 0.00016689300537109375 seconds
--- Binding consume in domain (wind, l, y)
    Completed in 6.914138793945312e-05 seconds
--- General Resource Balance for power in (l, q): initializing constraint, adding release(power, l, q)
    Completed in 0.000102996826171875 seconds
--- Binding release in domain (power, l, q)
    Completed in 0.00014019012451171875 seconds


In [7]:
m.y.show(True)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Check the model anytime, using m.show(), or object.show()

The first constraint is a general resource balance, generated for every resource at every spatiotemporal disposition at which an aspect regarding it is defined. 

Skip the True in .show() for a more concise set notation based print

In [8]:
m.power.show(True)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Process 

There are multiple ways to model this.

Whats most important, however, is the resource balance. The resource in the brackets is the basis resource. For a negative basis, multiply the resource by the negative factor or just use negation if that applies. 

In [9]:
m.wf = Process()
m.wf(m.power) == -1 * m.wind

Now set bounds on the extent to which the wind farm can operate. We are actually writing the following constraint: 

$\mathbf{opr} <= \phi \cdot \mathbf{cap}$

However, we know the capacity, so it is treated as a parameter. 

Note that the incoming values are normalized by default. Use norm = False to avoid that

In [10]:
m.wf.operate.prep(200, norm=False) <= [0.9, 0.8, 0.5, 0.7]
m.wf.show(True)

--- Binding operate in domain (wf, l, q)
--- Aspect (capacity) not defined, a variable will be created at l assuming y as the temporal index
    Completed in 0.0004208087921142578 seconds


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Alternatively, the capacity can be determined

In [11]:
# m.wf.capacity == True
# m.wf.operate <= [0.9, 0.8, 0.5, 0.7]
# m.wf.show(True)

Alternatively, 
a fixed capacity can be fixed

In [12]:
# m.wf.capacity == 200
# m.wf.operate <= [0.9, 0.8, 0.5, 0.7]
# m.wf.show(True)

Add a cost to the process operation. This implies that the cost of operating is variable in every quarter. In general:

$\dot{\mathbf{v}} == \theta \cdot \mathbf{v}$

In [13]:
m.wf.operate[m.usd.spend] == [4000, 4200, 4300, 3900]
m.spend.show(True)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [14]:
m.operate.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Note that printing can be achieved via the aspect [operate, spend, etc.] or the object

## Locating the process

The production streams are only generated once the process is places in some location. In this study, the only location is available is the default network.

In [15]:
m.network.locate(m.wf)

--- Assuming  wf capacity is unbounded in (l, y)
--- General Resource Balance for power in (l, q): adding produce(power, l, q, operate, wf)
    Completed in 0.00015974044799804688 seconds
--- General Resource Balance for wind in (l, y): adding expend(wind, l, y, operate, wf)
    Completed in 7.62939453125e-05 seconds
--- Mapping operate: (wf, l, q) → (wf, l, y)
    Completed in 0.000s


Alternatively,

In [16]:
# m.wf.locate(m.network)

# The Model

The model consists of the following:

1. A general resource balances for wind in (network, year) and power in (network, quarter)
2. Bounds on wind consumption [upper] and power release [lower], and wf operation [upper]
3. Conversion constraints, giving produced and expended resources based on operation
4. Calculation of spending USD in every quarter
5. Mapping constraints for operate: q -> y and spend: q -> y (generated after objective is set)


In [17]:
m.show(True)

# Mathematical Program for Program(example1)

<br><br>

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Mapping Constraints

<IPython.core.display.Math object>

## Optimize

For maximization use opt(False)

In [18]:
m.usd.spend.opt()

--- Mapping spend: (usd, l, q, operate, wf) → (usd, l, y)
    Completed in 0.000s
--- Generating Program(example1).mps
--- Creating gurobi model for Program(example1)
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-01
Read MPS format model from file Program(example1).mps
Reading time = 0.00 seconds
PROGRAM(EXAMPLE1): 25 rows, 21 columns, 51 nonzeros
--- Optimizing Program(example1) using gurobi
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 25 rows, 21 columns and 51 nonzeros
Model fingerprint: 0x40878474
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 4e+02]
Presolve removed 25 rows and 21 columns
Presolve time: 0.00s
Presolve: 

## The Solution

### The overall

In [19]:
m.sol()

# Solution for Program(example1)

<br><br>

## Objective

<IPython.core.display.Math object>

<br><br>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Individual

The individual solutions can be obtained as list as well (using aslist = True)